Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Prototype: Deploy Inception Model 

Prototype for Gallery scenario

### Login 

"!az login" or Data --> Connect to Azure

In [1]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "1baa1f3f-9b61-4f60-9da8-3a76ba416716",
  "isDefault": true,
  "name": "MSI",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "systemAssignedIdentity",
    "type": "servicePrincipal"
  }
}


### Load AML Workspace

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.subscription_id, ws.resource_group, ws.location)

Found the config file in: /home/nbuser/library/config.json
GalleryTestWorkSpace bc69d98c-7d2b-4542-88a4-f86eb4aea4a5 USEastGroup_AX eastus2


### Register InceptionV3 Models to MMS

It contains multiple models

In [3]:
from azureml.core.model import Model

models = []

new_model = Model.register(
    workspace = ws,
    model_path = "model/classify_image_graph_def.pb",
    model_name = "classify_image_graph_def")
models.append(new_model)

print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

Registering model classify_image_graph_def
Model id=classify_image_graph_def:7, name=classify_image_graph_def, created time=2018-10-11 23:24:54.322744+00:00


In [4]:
new_model = Model.register(
    workspace = ws,
    model_path = "model/imagenet_2012_challenge_label_map_proto.pbtxt",
    model_name = "imagenet_label_map")
models.append(new_model)

print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

Registering model imagenet_label_map
Model id=imagenet_label_map:6, name=imagenet_label_map, created time=2018-10-11 23:24:56.563854+00:00


In [5]:
new_model = Model.register(
    workspace = ws,
    model_path = "model/imagenet_synset_to_human_label_map.txt",
    model_name = "imagenet_s2h_label_map")
models.append(new_model)

print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

Registering model imagenet_s2h_label_map
Model id=imagenet_s2h_label_map:6, name=imagenet_s2h_label_map, created time=2018-10-11 23:25:04.630631+00:00


### Build Container Image 

In [6]:
import time
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script = "score.py",
    runtime = "python", 
    dependencies = ["app.py", "ui.html", "inception.py"],
    conda_file = "env.yml")

start = time.time()
image = Image.create(
    workspace = ws,
    name = "inception-v3-demo-image",
    models = models,
    image_config = image_config
)

image.wait_for_creation(show_output = True)

end = time.time()
print("Image creation latency: {} seconds".format(end-start))
print(image)

Creating image
Running.............................
SucceededImage creation operation finished for image inception-v3-demo-image:11, operation "Succeeded"
Image creation latency: 200.38700032234192 seconds
ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7f7abd2449b0>, name=inception-v3-demo-image, id=inception-v3-demo-image:11, tags={}, properties={}, version=11)


### Deploy Image to ACI

In [7]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

start = time.time()
aci_service = Webservice.deploy_from_image(
    workspace = ws,
    name = "inception-v3-demo-service",
    image = image,
    deployment_config = aci_config)
aci_service.wait_for_deployment(show_output = True)

end = time.time()
print("Service deployment latency: {} seconds".format(end-start))

Creating service
Running........................................
SucceededACI service creation operation finished, operation "Succeeded"
Service deployment latency: 242.53498077392578 seconds


### Run Inference

*TODO*: Make it UX friendly via either run a script with sample data or let us consume the UI page


In [8]:
# Get demo URL 
demo_uri = aci_service.scoring_uri.replace("/score", "/ui")
print("Try out the inceptionv2 model at {}".format(demo_uri))

Try out the inceptionv2 model at http://104.45.148.156:80/ui


### Cleanup Resources

Delete the web serivce, image, and models

In [9]:
aci_service.delete()
image.delete()
for m in models:
    m.delete()
print("Successfully deleted models, image and service.")

Successfully deleted models, image and service.
